In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.neighbors import NearestNeighbors
import ast


In [2]:
mentors = pd.read_csv('../generated_data/mentor.csv', usecols=lambda c: not c.startswith('Unnamed:'))
mentees = pd.read_csv('../generated_data/mentee.csv', usecols=lambda c: not c.startswith('Unnamed:'))

In [3]:
def ohe(mentors, mentees):
    mentees.drop(columns=['first_name', 'last_name'], inplace=True)
    mentees['profile_id'] = mentees['profile_id'].apply(lambda str: str.split('#')[1])    
    mentees.set_index('profile_id', inplace=True)
    
    mentees = mentees.join(mentees['time_zone'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum())
    mentees.drop(columns='time_zone', inplace=True)

    mentees = mentees.join(mentees['skill_rank'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum())
    mentees.drop(columns='skill_rank', inplace=True)

    mentees['tech_or_career'] = mentees['tech_or_career'].apply(ast.literal_eval)
    mentees = mentees.join(mentees['tech_or_career'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum())
    mentees.drop(columns='tech_or_career', inplace=True)


    mentees['skills'] = mentees['skills'].apply(ast.literal_eval)
    mentees = mentees.join(mentees['skills'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum())
    mentees.drop(columns='skills', inplace=True)

    mentors.drop(columns=['first_name', 'last_name'], inplace=True)
    mentors['profile_id'] = mentors['profile_id'].apply(lambda str: str.split('#')[1])    
    mentors.set_index('profile_id', inplace=True)
    mentors.index.rename('mentor_id', inplace=True)
    
    mentors = mentors.join(mentors['time_zone'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum())
    mentors.drop(columns='time_zone', inplace=True)

    mentors['skill_rank'] = mentors['skill_rank'].apply(ast.literal_eval)
    mentors = mentors.join(mentors['skill_rank'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum())
    mentors.drop(columns='skill_rank', inplace=True)

    mentors['tech_or_career'] = mentors['tech_or_career'].apply(ast.literal_eval)
    mentors = mentors.join(mentors['tech_or_career'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum())
    mentors.drop(columns='tech_or_career', inplace=True)


    mentors['skills'] = mentors['skills'].apply(ast.literal_eval)
    mentors = mentors.join(mentors['skills'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum())
    mentors.drop(columns='skills', inplace=True)
    return mentors, mentees

In [4]:
#ohe the dataset
#the driving force of selection is mentee to mentor <---- mentee like system
mentors, mentees = ohe(mentors, mentees)

In [5]:
mentors.head()

,Central,Eastern,Mountain,Pacific,Advanced,Beginner,Expert,Intermediate,Career Preparation,Technical,...,JavaScript,Microsoft Azure,MongoDB,Node.JS,PHP,PostgreSQL,Python,React,Ruby,SQL
mentor_id,,,,,,,,,,,,,,,,,,,,,
11996681750350,0,1,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,1,0
35370095635796,0,0,0,1,0,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,0
669953825711,1,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,1,0
42936362672176,0,0,0,1,0,0,0,1,1,1,...,0,1,0,0,1,0,0,0,0,0
63309130194498,1,0,0,0,1,1,1,1,0,1,...,1,0,0,0,0,1,1,0,0,0


In [6]:
mentees.head()

,Central,Eastern,Mountain,Pacific,Advanced,Beginner,Expert,Intermediate,Career Preparation,Technical,...,JavaScript,Microsoft Azure,MongoDB,Node.JS,PHP,PostgreSQL,Python,React,Ruby,SQL
profile_id,,,,,,,,,,,,,,,,,,,,,
44209311622914,1,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
56927275901552,0,0,0,1,0,0,1,0,1,1,...,0,0,1,0,0,0,1,0,0,0
1053163659625,0,0,0,1,0,1,0,0,1,1,...,0,1,0,0,0,0,0,0,0,0
26914565879809,0,0,0,1,0,0,1,0,1,1,...,0,0,1,0,0,0,0,0,0,1
37600750502571,1,0,0,0,0,0,0,1,0,1,...,0,0,1,1,0,0,0,0,0,0


In [7]:
#generate a fake review matrix
reviews = sp.sparse.random(mentees.shape[0], mentors.shape[0], density=0.1, random_state=42)
reviews = pd.DataFrame(reviews.todense(), index=mentees.index, columns=mentors.index)
reviews = reviews.melt(ignore_index=False ,value_name='rating')
reviews.head()

,mentor_id,rating
profile_id,,
44209311622914,11996681750350,0.0
56927275901552,11996681750350,0.0
1053163659625,11996681750350,0.0
26914565879809,11996681750350,0.0
37600750502571,11996681750350,0.0


In [8]:
#merge the reviews on profile id
combined = reviews.merge(mentees, left_index=True, right_index=True, how = 'left')
combined.head()

,mentor_id,rating,Central,Eastern,Mountain,Pacific,Advanced,Beginner,Expert,Intermediate,...,JavaScript,Microsoft Azure,MongoDB,Node.JS,PHP,PostgreSQL,Python,React,Ruby,SQL
profile_id,,,,,,,,,,,,,,,,,,,,,
1024429836615,11996681750350,0.000000,0,0,0,1,0,0,1,0,...,0,1,1,0,0,0,0,0,1,0
1024429836615,35370095635796,0.379006,0,0,0,1,0,0,1,0,...,0,1,1,0,0,0,0,0,1,0
1024429836615,669953825711,0.000000,0,0,0,1,0,0,1,0,...,0,1,1,0,0,0,0,0,1,0
1024429836615,42936362672176,0.000000,0,0,0,1,0,0,1,0,...,0,1,1,0,0,0,0,0,1,0
1024429836615,63309130194498,0.000000,0,0,0,1,0,0,1,0,...,0,1,1,0,0,0,0,0,1,0


In [9]:
#KNN model 
def create_model(df, neighbors_number):
    model = NearestNeighbors(n_neighbors=min(
        neighbors_number+1, df.shape[0]), metric='cosine')
    model.fit(df)
    return model

In [10]:
#Create model
knn_collab = create_model(combined.drop(columns=['mentor_id']), neighbors_number=10)
knn = create_model(mentors, neighbors_number=10)


random mentee#6967999171653,Raiden,Jones,"['Career Preparation', 'Technical']","['PHP', 'React', 'C++']",Advanced,Mountain

In [11]:
knn_collab_test = np.array([1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
                1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0])

knn_test = np.array([0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
                1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0])

In [12]:
#run a test on knn
#note that this returns MENTEES similar to the current mentee

dist_collab, indices_collab = knn_collab.kneighbors([knn_collab_test], 5)
#output indices of mentees that are similar to the current mentee
dist_knn, indices_knn = knn.kneighbors([knn_test], 5)

c:\Dev\python_envs\tensorflow\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Dev\python_envs\tensorflow\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [13]:
collab_mentor_list = []
for index in indices_collab:
    collab_mentor_list.append(mentors.loc[combined.iloc[index]['mentor_id']])

knn_list = []
for index in indices_knn:
    knn_list.append(mentors.loc[combined.iloc[index]['mentor_id']])

In [14]:
pd.set_option('display.max_columns', None)


In [17]:
collab_df = pd.DataFrame(collab_mentor_list[0])
collab_df

,Central,Eastern,Mountain,Pacific,Advanced,Beginner,Expert,Intermediate,Career Preparation,Technical,Android,Angular,Axios,C#,C++,CSS,Django,HTML,IOS,Java,JavaScript,Microsoft Azure,MongoDB,Node.JS,PHP,PostgreSQL,Python,React,Ruby,SQL
mentor_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8284601696244,1,0,0,0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0
15095524536358,0,0,0,1,0,1,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
25031268776199,1,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
51254476683696,1,0,0,0,1,1,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
66361726368093,0,0,1,0,1,0,1,1,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
knn_df = pd.DataFrame(knn_list[0])
knn_df

,Central,Eastern,Mountain,Pacific,Advanced,Beginner,Expert,Intermediate,Career Preparation,Technical,Android,Angular,Axios,C#,C++,CSS,Django,HTML,IOS,Java,JavaScript,Microsoft Azure,MongoDB,Node.JS,PHP,PostgreSQL,Python,React,Ruby,SQL
mentor_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
63121684978234,0,1,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1
45253072381561,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
63230246147329,0,0,0,1,0,0,1,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
42936362672176,0,0,0,1,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
8732351419446,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0


In [ ]:
combined[(combined['mentor_id'] == '63121684978234') & (combined['rating'] > 0)].rating.mean()
combined[(combined['mentor_id'] == '8284601696244') & (combined['rating'] > 0)].rating.mean()

0.5506750891461398

In [ ]:
mentors[(mentors['PHP'] == 1) & (mentors['React'] == 1) & (mentors['C++'] == 1)]

,Central,Eastern,Mountain,Pacific,Advanced,Beginner,Expert,Intermediate,Career Preparation,Technical,Android,Angular,Axios,C#,C++,CSS,Django,HTML,IOS,Java,JavaScript,Microsoft Azure,MongoDB,Node.JS,PHP,PostgreSQL,Python,React,Ruby,SQL
mentor_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
